# **Install Nessacary Packages**

In [4]:
pip install pinecone langchain_google_genai langchain_pinecone langchain_core


INFO: pip is looking at multiple versions of pinecone-client to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of langchain-pinecone to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 25.8 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 1.1.0
    Uninstalling pinecone-plugin-inference-1.1.0:
      Successfully uninstalled pinecone-plugin-inferenc

# **Fetch The API Keys**

In [5]:
from google.colab import userdata

PINECONE_API_KEY=userdata.get("PINECONE_API_KEY")
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
LANG_API_KEY=userdata.get("LANG_API_KEY")
OPENAI_API_KEY=userdata.get("OPENAI_KEY")

if PINECONE_API_KEY:
  print("API key fetched")
else:
  print("API key not fetched")

API key fetched


# **Initialize Pinecone with your API key**




In [6]:
from pinecone import Pinecone

pc=Pinecone(api_key=PINECONE_API_KEY)


# **List all previously Created indexes**

In [17]:
print(pc.list_indexes())


{'indexes': []}


# **Create a new Pinecone Index**

In [18]:
from pinecone import ServerlessSpec

index_name = "project-02-rag-wtih-langchain"

pc.create_index(
  name=index_name,
  dimension=768,
  spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

# **Initialize the Index**

In [19]:
index = pc.Index(index_name)


# **Check the current index statistics**

In [20]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# **Initialize Google Generative AI embeddings with the API key and model name**

In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY,model = "models/embedding-001")

# **Embed the query "Tell me my full name"**

In [22]:
vector= embeddings.embed_query("Tell me my full name")

# **Initialize the Pinecone vector store with the index and embeddings**

In [3]:
from langchain_pinecone import PineconeVectorStore

In [44]:
vector_store=PineconeVectorStore(index=index,embedding=embeddings,text_key='text')

# **Read and create Chunks from the input file**

In [73]:
file_path = "/content/MyData.txt"

with open(file_path, "r") as file:
    document_text = file.read()

chunks = document_text.split('\n')
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 8


# **Creating Document Objects from Text Chunks**

In [74]:
from langchain_core.documents import Document
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(chunks))]
documents = [
    Document(
        page_content=chunk,
        metadata={"source": "documents.txt", "chunk_index": idx},
    )
    for idx, chunk in enumerate(chunks)
]

In [75]:
for doc in documents:
  print(doc.page_content)

Full Name: Muhammad Abdul Wasay Abid
Age: 20 years old
Date of Birth: May 26, 2004
Father's Name: Hafiz Abid
Brother's Name: Uzair Abid
Current University: University of Central Punjab
Current Semester: 5th semester (3rd year)
Expected Graduation Year: 2026


In [76]:
vector_store.add_documents(documents=documents,ids=uuids)

['262bb58d-af64-4a26-a667-20325072ed8f',
 '8e3007d0-354f-467f-8b63-0022ba2c4dd1',
 '722b2a27-329e-41b5-9efb-8b75f6fedfec',
 'f34e22e3-4a06-4769-bee3-a300a60986c5',
 'bfb6c2cd-10c9-460d-bd03-7fc9289635df',
 'e4c12b00-d79d-4488-8c2e-f8bcb27f2c87',
 '51d48223-50ac-4d80-9cae-e363269ebd15',
 '03da4b05-fa1d-44c3-aa56-fd7546524c52']

# **Initialize the Gemini Model**

In [77]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(
model='gemini-1.5-flash',
temperature=0.9
)


# **Initializing the RetrievalQA Chain**

In [79]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

# **Running a Query with the RetrievalQA Chain**

In [81]:
query = """
what is my full name?
what is my brothers full name ?
"""

response = qa_chain.run(query)
print(response)

Your full name is Muhammad Abdul Wasay Abid.  Your brother's full name is Uzair Abid.

